In [1]:
import numpy as np
from numba import cuda

from evolution import world

%load_ext autoreload
%autoreload 2

In [ ]:
@cuda.jit('void(int32[:], int32[:])')
def base(x, y):
    i = cuda.threadIdx.x
    y[i] = x[i] + 1

t = np.arange(16).astype(np.int32)
z = np.zeros_like(t)
print(t.dtype)
print(z)
base[1, 20](t, z)
print(z)
base[1, 20](z, z)
print(z)

In [14]:
world.main()

float32 (4, 8, 3) float32 (4, 4) float32 (4, 8, 1)
